In [1]:
import pandas as pd
from training import *

In [5]:
# Example usage:
results = grid_search_flexsim(
    learning_rate_range=(0.0003, 0.0007, 0.0001),
    total_timesteps_range=(16000, 20000, 2000),
    gamma_range=(0.99, 1.01, 0.01),
    clip_range_range=(0.15, 0.20, 0.05)
)

df_results = pd.DataFrame(results)
df_results.to_csv('outputs/grid_search_results.csv', index=False)

Params: lr=0.0003, ts=16000, gamma=0.99, clip=0.15
Reward: -60.1 +/- 16.91
------------------------
Params: lr=0.0003, ts=16000, gamma=0.99, clip=0.2
Reward: -59.233 +/- 15.253
------------------------
Params: lr=0.0003, ts=16000, gamma=1.0, clip=0.15
Reward: -41.917 +/- 15.671
------------------------
Params: lr=0.0003, ts=16000, gamma=1.0, clip=0.2
Reward: -42.033 +/- 14.15
------------------------
Params: lr=0.0003, ts=16000, gamma=1.01, clip=0.15
Reward: -50.417 +/- 11.403
------------------------
Params: lr=0.0003, ts=16000, gamma=1.01, clip=0.2
Reward: -61.633 +/- 11.371
------------------------
Params: lr=0.0003, ts=18000, gamma=0.99, clip=0.15
Reward: -36.7 +/- 11.453
------------------------
Params: lr=0.0003, ts=18000, gamma=0.99, clip=0.2
Reward: -61.25 +/- 13.411
------------------------
Params: lr=0.0003, ts=18000, gamma=1.0, clip=0.15
Reward: -55.7 +/- 13.246
------------------------
Params: lr=0.0003, ts=18000, gamma=1.0, clip=0.2
Reward: -38.6 +/- 9.721
----------------

In [2]:
df_results = pd.read_csv('outputs/grid_search_results.csv')
df_results.sort_values(by='mean_reward', ascending=False).head(10)

,lr,ts,gamma,clip,mean_reward,std_reward
32,0.0005,16000,0.99,0.15,-44.800000,12.045469
22,0.0004,16000,1.00,0.15,-45.033333,13.763438
1,0.0003,14000,0.98,0.20,-45.133333,13.814807
47,0.0006,16000,1.00,0.20,-45.200000,15.273943
17,0.0004,14000,1.00,0.20,-45.433333,12.922031
38,0.0006,14000,0.99,0.15,-45.666667,18.225135
33,0.0005,16000,0.99,0.20,-45.766667,12.742798
30,0.0005,16000,0.98,0.15,-45.933333,12.974162
18,0.0004,16000,0.98,0.15,-46.133333,18.245974
34,0.0005,16000,1.00,0.15,-46.400000,15.409088


In [4]:
df_results['lr'].unique()

array([0.0003, 0.0004, 0.0005, 0.0006])